In [ ]:
# Import dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sqlalchemy import create_engine
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# load data back into a dataframe from the database
mental_df = pd.read_sql_table('pre_encoded_survey', 'sqlite:///mental_health.db').drop(columns=['index'])
mental_df

In [ ]:
mental_df.dtypes

In [ ]:
# Generate our categorical variable list
mental_cat = mental_df.dtypes[mental_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
mental_df[mental_cat].nunique()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(mental_df[mental_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(mental_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
mental_df = mental_df.merge(encode_df,left_index=True, right_index=True)
mental_df = mental_df.drop(mental_cat,1)
mental_df.head()

In [ ]:
# Create the engine
engine = create_engine("sqlite:///mental_health.db", echo=False)

In [ ]:
# Use pandas to_sql to write records stored in a dataframe to a SQL database

# Write dataframe of all survey questions to a sqlite table
mental_df.to_sql('encoded_survey', con=engine, if_exists='replace')
#engine.execute("SELECT * FROM encoded_survey").fetchall()

In [ ]:
# load data back into a dataframe from the database
mental_df = pd.read_sql_table('encoded_survey', 'sqlite:///mental_health.db').drop(columns=['index'])
mental_df

In [ ]:
# load data back into a dataframe from the database
questions_df = pd.read_sql_table('pre_encoded_questions', 'sqlite:///mental_health.db').drop(columns=['index'])
questions_df

In [ ]:
# Split the dataset into target and feature sets
y = mental_df['25']  # replace 'target_variable' with the name of your target variable
X = mental_df.drop(['25'], axis=1)  # drop the target variable from the feature set


In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

In [ ]:
# Fitting the train model
rf_model = rf_model.fit(X_train, y_train)

In [ ]:
import joblib

In [ ]:
# Save the trained model to disk
joblib.dump(rf_model, 'rf_model.joblib')

# Check the size of the saved model file
import os
model_size = os.path.getsize('rf_model.joblib')
print(f"Model size: {model_size} bytes")

In [ ]:
import time
start_time = time.time()
rf_model.fit(X_train, y_train)
end_time = time.time()

# Calculate the elapsed time in seconds
elapsed_time_seconds = end_time - start_time
print(f"Training time: {elapsed_time_seconds} seconds")

In [ ]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test)
predictions

In [ ]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

In [ ]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Define the target set.
y = mental_df["25"]

# Define the features set.
X = mental_df.drop(columns=["25","25"])

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Scale the features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Reshape the features to 3D tensor format
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Build the RNN model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
# Evaluate the model on the test set
score, acc = model.evaluate(X_test, y_test, batch_size=32)

In [ ]:
# Print the accuracy of the model
print('Test accuracy:', acc)

In [ ]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
#Naive Byes

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Define the target set.
#dependent variable
y = mental_df["25"]

# Define the features set.
#independent variable
X = mental_df.drop(columns=["25","25"])

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a Naive Bayes model
nb_model = GaussianNB()

In [ ]:
# Train the model on the training set
nb_model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = nb_model.predict(X_test)


In [ ]:
# Evaluate the model's accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
# Print the accuracy of the model
print('Test accuracy:', acc)

In [ ]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
# Define the target set.
y = mental_df["25"]

# Define the features set.
X = mental_df.drop(columns=["25","25"])

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create an SVM model
svm_model = SVC(kernel='linear')











In [ ]:
# Train the model on the training set
svm_model.fit(X_train, y_train)


In [ ]:
# Make predictions on the test set
y_pred = svm_model.predict(X_test)

In [ ]:
# Evaluate the model's accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy of the model
print('Test accuracy:', accuracy)

In [ ]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# Define the target set.
y = mental_df["25"]

# Define the features set.
X = mental_df.drop(columns=["25","25"])

In [ ]:
# Preprocess the data for the CNN model
image_width = 64
image_height = 64
X = np.array(X)
X = X.reshape(-1, image_width, image_height, 1)
X = X.astype('float32')
X /= 255

In [ ]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
import sklearn
print(sklearn.__version__)